In [1]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush'
 'Gerhard Schroeder' 'Hugo Chavez' 'Junichiro Koizumi' 'Tony Blair']
(1348, 62, 47)


### **Que hace esta celda?**
Esta celda descarga y carga el conjunto de datos "Labeled Faces in the Wild (LFW)", un dataset muy usado en aprendizaje automatico para experimentos de reconocimiento facial.
Luego, imprime los nombres de las personas incluidas y las dimensiones del conjunto de imagenes.

### **Explicacion paso a paso**
- ```fetch_lfw_people``` carga el dataset Labeled Faces in the Wild (LFW), un conjunto de imagenes de rostros usado para reconocimiento facial.

- El parámetro ```min_faces_per_person=60``` filtra el conjunto para incluir solo personas con al menos 60 fotografías, evitando clases con pocos ejemplos.

- ```faces.target_names``` muestra los nombres de las personas (clases).

- ```faces.images.shape``` indica las dimensiones del dataset, por ejemplo ```(1348, 62, 47)``` → 1348 imágenes de tamaño 62×47 píxeles.

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

### **Que hace esta celda?**
Esta celda crea una cuadrícula de subgráficos para **inspeccionar visualmente** algunas imágenes del dataset LFW y verificar que la carga de datos sea correcta. Al mostrar las primeras 15 caras con su etiqueta, nos formamos una idea rápida de la **variabilidad** entre clases y de la **calidad** de las imágenes antes de continuar con el modelado.


### **Explicacion paso a paso**
- Se importa ```matplotlib.pyplot``` para crear gráficos y se usa %matplotlib inline para mostrar las imágenes dentro del notebook.

- ```plt.subplots(3, 5)``` crea una cuadrícula de 3 filas y 5 columnas de ejes.

- En el bucle ```for```, se muestran las primeras 15 imágenes del dataset (```faces.images[i]```) en escala de grises (```cmap='bone'```).

- Se eliminan los ejes (```xticks=[]```, ```yticks=[]```) y se muestra el nombre de la persona debajo de cada imagen.

In [3]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components=150, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

### **Que hace esta celda?**
En esta celda se construye un **modelo de clasificación de rostros** combinando dos etapas:  
una **reducción de dimensionalidad** mediante *PCA* y una **clasificación** con una *Máquina de Vectores de Soporte (SVM)*.  
Esto permite simplificar los datos (imágenes con miles de píxeles) y mejorar el rendimiento del clasificador.


### **Explicacion paso a paso**
- ```PCA(n_components=150)``` reduce las imágenes a 150 componentes principales, conservando la mayor parte de la varianza (información relevante).

- ```whiten=True``` normaliza las componentes para que tengan varianza unitaria, mejorando el desempeño del modelo.

- ```SVC(kernel='rbf')``` crea un clasificador SVM con kernel radial, adecuado para datos no lineales.

- ```class_weight='balanced'``` ajusta automáticamente el peso de cada clase según su frecuencia.

- ```make_pipeline(pca, svc)``` une ambos pasos en un flujo de trabajo (pipeline), aplicando primero el PCA y luego el SVM.

In [4]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target, random_state=42)

### **Que hace esta celda?**
Esta celda separa el conjunto de imágenes en **datos de entrenamiento y prueba**, lo que permite evaluar el rendimiento real del modelo en datos no vistos.  
El entrenamiento sirve para ajustar los parámetros del modelo, mientras que la prueba mide su capacidad de generalización.


### **Explicacion paso a paso**
- ```train_test_split``` divide los datos (```faces.data```) y las etiquetas (```faces.target```) en dos subconjuntos.

- Por defecto, usa 75 % para entrenamiento y 25 % para prueba.

- ```random_state=42``` fija la semilla aleatoria para garantizar que la división sea reproducible.

In [5]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

CPU times: total: 55.1 s
Wall time: 43.4 s
{'svc__C': 5, 'svc__gamma': 0.001}


### **Que hace esta celda?**
Esta celda realiza una **búsqueda exhaustiva de hiperparámetros** para optimizar el rendimiento del modelo SVM dentro del pipeline.  
El objetivo es encontrar los valores de `C` y `gamma` que producen la mejor precisión en validación cruzada.

### **Explicacion paso a paso**
- ```param_grid``` define las combinaciones de hiperparámetros que se probarán:

    - ```C```: controla la penalización por error (regularización).

    - ```gamma```: define la influencia de cada punto en el kernel RBF.

- Los prefijos ```svc__``` indican que los parámetros pertenecen al SVC dentro del pipeline.

- ```GridSearchCV``` entrena el modelo con todas las combinaciones posibles usando validación cruzada.

- ```grid.best_params_``` muestra los valores óptimos encontrados.

- ```%time``` mide el tiempo de ejecución del proceso.

In [6]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

### **Que hace esta celda?**
Esta celda utiliza el **mejor modelo encontrado por la búsqueda en malla (GridSearchCV)** para hacer predicciones sobre los datos de prueba.  
Así se evalúa el rendimiento final del clasificador con los hiperparámetros óptimos.


### **Explicacion paso a paso**
- ```grid.best_estimator_``` recupera el modelo con la mejor combinación de parámetros (```C``` y ```gamma```) hallada en la búsqueda.

- ```model.predict(Xtest)``` aplica ese modelo optimizado para predecir las etiquetas de las imágenes de prueba.

- El resultado ```yfit``` contiene las clases predichas para cada rostro en ```Xtest```.

In [7]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')

### **Que hace esta celda?**
Esta celda muestra una **muestra de imágenes de prueba** junto con sus **predicciones** para observar visualmente el desempeño del modelo.  
Las etiquetas correctas aparecen en **negro** y las incorrectas en **rojo**, lo que permite identificar fácilmente los errores de clasificación.


### **Explicacion paso a paso**
- ```plt.subplots(4, 6)``` crea una cuadrícula de 24 imágenes.

- Cada imagen de ```Xtest``` se reconvierte a su forma original (62×47 píxeles).

- El nombre predicho (```yfit[i]```) se muestra como etiqueta;
si coincide con la etiqueta real (```ytest[i]```), el texto es negro, y si no, rojo.

- Esto proporciona una evaluación visual rápida de los aciertos y errores del modelo.

In [8]:

from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

                   precision    recall  f1-score   support

     Ariel Sharon       0.65      0.87      0.74        15
     Colin Powell       0.83      0.88      0.86        68
  Donald Rumsfeld       0.70      0.84      0.76        31
    George W Bush       0.97      0.80      0.88       126
Gerhard Schroeder       0.76      0.83      0.79        23
      Hugo Chavez       0.93      0.70      0.80        20
Junichiro Koizumi       0.86      1.00      0.92        12
       Tony Blair       0.82      0.98      0.89        42

         accuracy                           0.85       337
        macro avg       0.82      0.86      0.83       337
     weighted avg       0.86      0.85      0.85       337



### **Que hace esta celda?**
Esta celda genera un **informe de clasificación** que resume el rendimiento del modelo mediante métricas estadísticas clave.  
Permite identificar qué tan bien el modelo reconoce cada rostro.

### **Explicacion paso a paso**
- ```classification_report``` compara las etiquetas reales (```ytest```) con las predichas (```yfit```).

- Muestra métricas por clase:

    - **Precisión (precision)**: proporción de predicciones correctas entre las positivas.

    - **Exhaustividad (recall)**: capacidad del modelo para detectar todas las instancias reales.

    - **F1-score**: media armónica entre precisión y recall.

- ```target_names``` asocia los resultados con los nombres reales de las personas.

In [9]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)

<Axes: >

### **Que hace esta celda?**
Esta celda muestra una **matriz de confusión**, una herramienta visual para evaluar qué tan bien el modelo clasifica cada rostro.  
Permite ver fácilmente en qué clases se confunde el clasificador.


### **Explicacion paso a paso**
- ```confusion_matrix(ytest, yfit)``` genera una tabla donde cada fila representa las clases reales y cada columna las predicciones.

- ```sns.heatmap``` visualiza esa matriz como un mapa de calor, mostrando con números (```annot=True```) la cantidad de aciertos y errores.

- Las etiquetas de los ejes (```xticklabels``` y ```yticklabels```) corresponden a los nombres de las personas del dataset.

- Los valores altos en la diagonal indican buen rendimiento, mientras que los fuera de ella muestran confusiones entre clases.


## **Dataset load_digits**

### **Celda 1 — Carga del dataset Digits**
Cargamos el conjunto **Digits** y mostramos los nombres de clase y la forma de las imágenes.


In [10]:
from sklearn.datasets import load_digits
digits = load_digits()
print(getattr(digits, "target_names", list(range(10))))
print(digits.images.shape)


[0 1 2 3 4 5 6 7 8 9]
(1797, 8, 8)


### Celda 2 — Visualización de ejemplos del dataset
Mostramos una cuadrícula de imágenes para inspección visual rápida (mismas convenciones que antes).


In [12]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(digits.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=str(digits.target[i]))


### Celda 3 — Creación del modelo con PCA y SVM
Construimos el **pipeline** con PCA (blanqueado) + SVC (RBF), igual que antes.


In [13]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components=40, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)


### Celda 4 — División del conjunto de datos
Separación en **entrenamiento** y **prueba** con semilla fija (por defecto 75/25).


In [14]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target, random_state=42)


### Celda 5 — Búsqueda de hiperparámetros con GridSearchCV
Optimizamos `C` y `gamma` del SVC dentro del mismo pipeline (mismo grid y prefijos).


In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)


CPU times: total: 28.3 s
Wall time: 27.6 s
{'svc__C': 5, 'svc__gamma': 0.005}


### Celda 6 — Predicción con el mejor modelo
Usamos el mejor estimador hallado por la búsqueda para predecir en el set de prueba.


In [16]:
model = grid.best_estimator_
yfit = model.predict(Xtest)


### Celda 7 — Visualización de predicciones del modelo
Mostramos una cuadrícula de imágenes de prueba con la **predicción**; negro si acierta y rojo si falla.


In [17]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(8, 8), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(str(yfit[i]),
                   color='black' if yfit[i] == ytest[i] else 'red')


### Celda 8 — Evaluación con classification_report
Informe con **precision**, **recall** y **f1-score** por clase y promedios.


In [18]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        37
           2       1.00      1.00      1.00        38
           3       1.00      0.96      0.98        46
           4       1.00      1.00      1.00        55
           5       0.97      1.00      0.98        59
           6       1.00      0.98      0.99        45
           7       1.00      0.98      0.99        41
           8       0.95      1.00      0.97        38
           9       0.98      0.98      0.98        48

    accuracy                           0.99       450
   macro avg       0.99      0.99      0.99       450
weighted avg       0.99      0.99      0.99       450



### Celda 9 — Matriz de confusión
Mapa de calor para ver aciertos (diagonal) y confusiones entre dígitos.


In [19]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=getattr(digits, "target_names", list(range(10))),
            yticklabels=getattr(digits, "target_names", list(range(10))))


<Axes: >